In [1]:
import pandas as pd

In [2]:
PATH = '../../data/'

In [42]:
reviews = pd.read_csv(PATH + 'processed/' + 'sg_coffee_reviews_final.csv')
reviews['date'] = pd.to_datetime(reviews['date'])
reviews.head()

,shop,userid,userinfo,rating,date,photos,username,language,text
0,nylon-coffee-roasters-singapore,AbfS_oXF8H6HJb5jFqhrLw,{'link': '/user_details?userid=AbfS_oXF8H6HJb5...,5.0,2016-04-11,[],Paco G.,en,"A comfortable, no-frills coffee shop right in ..."
1,nylon-coffee-roasters-singapore,ymiohz-DlNLHGAlXbVhw5A,{'link': '/user_details?userid=ymiohz-DlNLHGAl...,4.0,2019-11-11,[{'src': 'https://s3-media0.fl.yelpcdn.com/bph...,Sudipto G.,en,A hard to find coffee place tucked away in the...
2,nylon-coffee-roasters-singapore,-g3XIcCb2b-BD0QBCcq2Sw,{'link': '/user_details?userid=-g3XIcCb2b-BD0Q...,4.0,2019-12-13,[{'src': 'https://s3-media0.fl.yelpcdn.com/bph...,Lisa I.,en,Great little coffee shop that's in the middle ...
3,nylon-coffee-roasters-singapore,gfyW0vuJls8ARoHmogfcmw,{'link': '/user_details?userid=gfyW0vuJls8ARoH...,4.0,2020-02-06,[],Marilyn T.,en,This place came highly recommended by a friend...
4,nylon-coffee-roasters-singapore,jkr3YVCEoRNKkxinUZpW6A,{'link': '/user_details?userid=jkr3YVCEoRNKkxi...,3.0,2015-02-12,[],Joanne G.,en,My great auntie has lived at Everton Park for ...


In [43]:
MIN_REVIEWS = 3

In [44]:
reviews['num_reviews'] = reviews.groupby(['userid'])['rating'].transform('count')
reviews = reviews.loc[reviews['num_reviews'] >= MIN_REVIEWS]
reviews.head()

,shop,userid,userinfo,rating,date,photos,username,language,text,num_reviews
0,nylon-coffee-roasters-singapore,AbfS_oXF8H6HJb5jFqhrLw,{'link': '/user_details?userid=AbfS_oXF8H6HJb5...,5.0,2016-04-11,[],Paco G.,en,"A comfortable, no-frills coffee shop right in ...",11
1,nylon-coffee-roasters-singapore,ymiohz-DlNLHGAlXbVhw5A,{'link': '/user_details?userid=ymiohz-DlNLHGAl...,4.0,2019-11-11,[{'src': 'https://s3-media0.fl.yelpcdn.com/bph...,Sudipto G.,en,A hard to find coffee place tucked away in the...,48
4,nylon-coffee-roasters-singapore,jkr3YVCEoRNKkxinUZpW6A,{'link': '/user_details?userid=jkr3YVCEoRNKkxi...,3.0,2015-02-12,[],Joanne G.,en,My great auntie has lived at Everton Park for ...,8
5,nylon-coffee-roasters-singapore,K4ZeG8WAPfTS1AjD5GoJTw,{'link': '/user_details?userid=K4ZeG8WAPfTS1Aj...,4.0,2014-11-15,[{'src': 'https://s3-media0.fl.yelpcdn.com/bph...,Tzeko M.,en,This coffee shop's so hipster it hurts. But I ...,20
7,nylon-coffee-roasters-singapore,O2itSY4F2jnPRXaxdVGOkQ,{'link': '/user_details?userid=O2itSY4F2jnPRXa...,5.0,2015-11-30,[{'src': 'https://s3-media0.fl.yelpcdn.com/bph...,Wan Qi T.,en,"My first reaction when checked in here, ""OMG S...",6


In [45]:
print('Number of unique users: {}'.format(len(reviews.userid.unique())))
print('Number of unique shops: {}'.format(len(reviews.shop.unique())))
print('Number of ratings: {}'.format(len(reviews)))
sparsity = 1 - len(reviews) / (len(reviews.shop.unique()) * len(reviews.userid.unique()))
print('Sparsity of dataset: {}'.format(sparsity))

Number of unique users: 531
Number of unique shops: 856
Number of ratings: 4050
Sparsity of dataset: 0.991089814668145


### Leave One Out

In [40]:
reviews = reviews.sort_values(['userid', 'date'], ascending=[True,True])
val = reviews.groupby(['userid']).tail(2).groupby(['userid']).head(1).reset_index(drop=True)
test = reviews.groupby(['userid']).tail(1).reset_index(drop=True)
train = reviews[reviews.groupby(['userid']).cumcount(ascending=False) > 1].reset_index(drop=True)

In [49]:
print('Number of ratings in train set: {}'.format(train.shape[0]))
print('Number of ratings in val set: {}'.format(val.shape[0]))
print('Number of ratings in test set: {}'.format(test.shape[0]))

Number of ratings in train set: 2988
Number of ratings in val set: 531
Number of ratings in test set: 531


In [ ]:
train.to_csv(PATH + 'processed/' + 'train_loo.csv', index=False)
val.to_csv(PATH + 'processed/' + 'val_loo.csv', index=False)
test.to_csv(PATH + 'processed/' + 'test_loo.csv', index=False)